In [2]:
import numpy as np
import pandas as pd
from IPython.display import display, HTML

In [3]:
mappings = pd.read_csv('Module-Mappings.csv')

In [4]:
mappings = mappings.fillna('INVALID')

In [5]:
input_dict1 = {'Location_type': 'regions', 'Location': ['Europe'], 'Departments': {'MA': 2, 'CS': 3}}
input_dict2 = {'Location_type': 'Countries', 'Location': ['Singapore'], 'Departments': {'LSM': 2, 'ST': 3}}
input_dict3 = {'Location_type': 'Universities', 'Location': ['Stanford University'], 'Departments': {'MNO': 3}}

In [8]:
input_dict1

{'Location_type': 'regions',
 'Location': ['Europe'],
 'Departments': {'MA': 2, 'CS': 3}}

In [6]:
def dict_merger(dict1,dict2):
    for region in dict2:
        if region not in dict1:
            dict1[region] = dict2[region]
            continue
        for country in dict2[region]:
            if country not in dict1[region]:
                dict1[region][country] = dict2[region][country]
                continue
            # Doesn't matter if we overwrite since the data is the same if they exist in both
            for uni in dict2[region][country]:
                dict1[region][country][uni] = dict2[region][country][uni]

    return dict1

def main(input_dict):
    if input_dict['Location_type'].lower() == 'regions':
        areas = mappings[mappings['Continent'].isin(input_dict['Location'])]
    elif input_dict['Location_type'].lower() == 'countries':
        areas = mappings[mappings['Country'].isin(input_dict['Location'])]
    elif input_dict['Location_type'].lower() == 'universities':
        areas = mappings[mappings['Partner University'].isin(input_dict['Location'])]
    else:
        raise ValueError("Value of Location_type MUST be either 'regions', 'countries' or 'universities'")
    departments = list(input_dict['Departments'].keys())
    truthy = (areas['Department 1'] == departments[0]) & (areas['Level'] == input_dict['Departments'][departments[0]])
    for key in departments[1:]:
        truthy = truthy | (areas['Department 1'] == key) & (areas['Level'] == input_dict['Departments'][key])
    levels = areas[truthy]

    unique_unis = np.unique(levels['Partner University'])

    output_dict = {}
    for uni in unique_unis:
        uni_reference = levels[levels['Partner University'] == uni]
        region = uni_reference['Continent'].iloc[0]
        country = uni_reference['Country'].iloc[0]
        if region not in output_dict:
            output_dict[region] = {country: {}}
        if country not in output_dict[region]:
            output_dict[region][country] = {uni: {}}
        if uni not in output_dict[region][country]:
            output_dict[region][country][uni] = {}
        modules = uni_reference['NUS Module 1']
        unique_modules = np.unique(modules)
        for module in unique_modules:
            row = uni_reference[uni_reference['NUS Module 1'] == module]
            PU_title = row['PU Module 1 Title'].iloc[0]
            PU_code = row['PU Module 1'].iloc[0]
            output_dict[region][country][uni][module] = {'PU Module Title': PU_title, 'PU Module Code': PU_code}
    return output_dict

In [7]:
o1 = main(input_dict1)
o2 = main(input_dict2)
o3 = main(input_dict3)

o = dict_merger(o1, o2)
o = dict_merger(o,o3)
o

{'Europe': {'Denmark': {'Aarhus University': {'CS3223': {'PU Module Title': 'Deployment and Uses of Databases',
     'PU Module Code': 'CS220171U012'},
    'CS3226': {'PU Module Title': 'The Web of Things',
     'PU Module Code': 'CS220171U013'},
    'CS3230': {'PU Module Title': 'Foundations of Algorithms and Data Structures',
     'PU Module Code': '220171U002'},
    'CS3235': {'PU Module Title': 'IT Security and Cryptography in Practice',
     'PU Module Code': '220164U003'},
    'CS3240': {'PU Module Title': 'Designing Interactive Technologies',
     'PU Module Code': '520172U016'},
    'CS3244': {'PU Module Title': 'Machine Learning',
     'PU Module Code': '220171U007'}},
   'Technical University of Denmark': {'CS3210': {'PU Module Title': 'Parallel Computer Systems',
     'PU Module Code': '2258'},
    'CS3234': {'PU Module Title': 'Logical Systems and Logic Programming',
     'PU Module Code': '2156'},
    'CS3240': {'PU Module Title': 'Personal Data Interaction for Mobile and 